In [1]:
import numpy as np, matplotlib.pyplot as plt
import matelms as me
import test
from scipy.sparse.linalg import gmres, bicgstab, LinearOperator

In [2]:
PI = 3.14159265359
b, eta = 1.0, 0.1
xij,yij,zij = np.array([-2., 0., 0.])

In [3]:
##diagonal matrix elements
g2s = np.full(5, 3./(20*PI*eta*b))
g3t = np.full(3, 1./(2*PI*eta*b))
g3a = np.full(5, 3./(2*PI*eta*b))
g3s = np.full(7, 6./(7*PI*eta*b))

GoHH = np.diag(np.block([g2s,g3t,g3a,g3s]))

##construct two-body GHH, dimension 40x40
def GHH(xij,yij,zij):
    return np.block([[GoHH, me.GHH(xij,yij,zij, b,eta)],
                    [me.GHH(-xij,-yij,-zij, b,eta), GoHH]])

In [4]:
def GHHFH(FH):
    return np.concatenate([(test.GoHHFH(b,eta, FH[:20]) + test.GHHFH(xij,yij,zij, b,eta, FH[20:]))
                     ,(test.GHHFH(-xij,-yij,-zij, b,eta, FH[:20])+ test.GoHHFH(b,eta, FH[20:]))])

In [5]:
np.dot(GHH(xij,yij,zij),np.ones(40)).shape

(40,)

In [6]:
rhs = np.dot(GHH(xij,yij,zij),np.ones(40)) 
Ax = LinearOperator((40,40),matvec=GHHFH) ##Ax as linear operator

In [7]:
%%time
soln, exitCode = bicgstab(Ax, rhs)

CPU times: user 148 ms, sys: 3.14 ms, total: 151 ms
Wall time: 148 ms


In [8]:
soln

array([0.99998402, 1.00001962, 1.00003856, 1.00003246, 0.99997002,
       0.99999996, 0.99999996, 0.99999996, 1.00000045, 0.99999905,
       1.00000048, 0.99999918, 0.99999918, 0.99998936, 0.99999987,
       0.99998638, 1.0000092 , 1.0000027 , 1.00000405, 1.00000624,
       0.99984945, 1.00000765, 0.99998436, 1.00007455, 0.99997655,
       0.99999996, 0.99999996, 0.99999996, 0.99999963, 0.99999988,
       0.9999995 , 1.00000057, 1.00000098, 0.99999545, 1.00000012,
       0.99998655, 1.00000655, 1.00000255, 1.00000387, 1.0000062 ])

In [9]:
x0 = rhs/np.concatenate([test.GoHHFH(b,eta, np.ones(20)), test.GoHHFH(b,eta, np.ones(20))])

In [10]:
%%time
soln, exitCode = bicgstab(Ax, rhs, x0)

CPU times: user 159 ms, sys: 917 µs, total: 160 ms
Wall time: 158 ms


In [11]:
soln

array([0.99980928, 0.99998811, 1.00002416, 1.00009398, 0.99998818,
       1.        , 1.        , 1.        , 1.00001541, 1.00000145,
       0.99999937, 0.9999916 , 0.99999844, 0.99998789, 0.99999997,
       1.00000063, 1.00000604, 1.00000009, 1.00000002, 0.99999984,
       0.99979574, 1.00000583, 0.99996978, 1.00007223, 1.00000601,
       1.        , 1.        , 1.        , 0.9999863 , 1.00000025,
       1.00000059, 1.00000797, 1.00000161, 1.0000115 , 0.99999981,
       1.0000005 , 0.99999456, 0.99999957, 1.00000014, 0.99999987])

In [12]:
##maybe x0 helps for many particles and time steps

In [13]:
np.zeros(3)

array([0., 0., 0.])

In [14]:
np.zeros([3])

array([0., 0., 0.])

In [15]:
test.KHHVH(xij,yij,zij, b,eta, np.ones(20))

array([-0.01083333,  0.00125   ,  0.00125   ,  0.01791667,  0.0125    ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.125     ,
       -0.125     , -0.0625    ,  0.0625    ,  0.00214286,  0.00369048,
        0.00369048,  0.00488095,  0.00595238, -0.00276786, -0.00092262])

### Check whether all of them match with previous results! - Yes, seems to work, unless I have made a mistake making the below trivial

In [16]:
LL = np.random.normal(np.ones(3))
HH = np.random.normal(np.ones(20))

In [17]:
np.allclose(test.G1s1sF(xij,yij,zij, b,eta, LL), np.dot(me.G1s1s(xij,yij,zij, b,eta), LL))

True

In [18]:
np.allclose(test.G2a1sF(xij,yij,zij, b,eta, LL), np.dot(me.G2a1s(xij,yij,zij, b,eta), LL))

True

In [19]:
np.allclose(test.G1s2aT(xij,yij,zij, b,eta, LL), np.dot(me.G1s2a(xij,yij,zij, b,eta), LL))

True

In [20]:
np.allclose(test.G2a2aT(xij,yij,zij, b,eta, LL), np.dot(me.G2a2a(xij,yij,zij, b,eta), LL))

True

In [21]:
np.allclose(test.GH1sF(xij,yij,zij, b,eta, LL), np.dot(me.GH1s(xij,yij,zij, b,eta), LL))

True

In [22]:
np.allclose(test.GH2aT(xij,yij,zij, b,eta, LL), np.dot(me.GH2a(xij,yij,zij, b,eta), LL))

True

In [23]:
np.allclose(test.GHHFH(xij,yij,zij, b,eta, HH), np.dot(me.GHH(xij,yij,zij, b,eta), HH))

True

In [24]:
np.allclose(test.KHHVH(xij,yij,zij, b,eta, HH), np.dot(me.KHH(xij,yij,zij, b,eta), HH))

True

In [25]:
from linearSolve import linearSolve_direct as LS
ls = LS(b, 2, eta)

In [26]:
np.allclose(test.GoHHFH(b,eta, HH), np.dot(ls.GoHH,HH))

True

In [27]:
np.allclose(test.KoHHVH(b,eta, HH), np.dot(ls.KoHH, HH))

True

In [28]:
np.allclose(test.G1sHFH(xij,yij,zij, b,eta, HH), np.dot(me.G1sH(xij,yij,zij, b,eta), HH))

True

In [29]:
np.allclose(test.G2aHFH(xij,yij,zij, b,eta, HH), np.dot(me.G2aH(xij,yij,zij, b,eta), HH))

True

In [30]:
np.allclose(test.K1sHVH(xij,yij,zij, b,eta, HH), np.dot(me.K1sH(xij,yij,zij, b,eta), HH))

True

In [31]:
np.allclose(test.K2aHVH(xij,yij,zij, b,eta, HH), np.dot(me.K2aH(xij,yij,zij, b,eta), HH))

True

In [32]:
ff = np.array([1,0,0])

In [33]:
test.GH1sF(xij,yij,zij, b,eta, ff)

array([ 0.11936621,  0.        ,  0.        , -0.0596831 ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.09662979,  0.        ,
        0.        , -0.04831489,  0.        ,  0.        ,  0.        ])

In [34]:
np.dot(me.GH1s(xij,yij,zij, b,eta), ff)

array([ 0.11936621,  0.        ,  0.        , -0.0596831 ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.09662979,  0.        ,
        0.        , -0.04831489,  0.        ,  0.        ,  0.        ])

In [35]:
np.allclose(test.GH1sF(xij,yij,zij, b,eta, LL), np.dot(me.GH1s(xij,yij,zij, b,eta), LL))

True